In [1]:
using GaussianProcesses, Plots, SumProductNetworks, StatsFuns, Distributions
using PlotRecipes
using TestImages, Images
import SumProductNetworks.add!

#### include local code

In [41]:
include("utilFunctions.jl")
include("dataTypes.jl")
include("dataTypeUtilFunctions.jl")
include("computationFunctions.jl")
include("regionGraph.jl")
include("regionGraphUtils.jl")
include("gpUtils.jl")

mcmcKernel (generic function with 1 method)

# Load data

In [3]:
datapath = "../data/clean/motor.csv"
(data, header) = readcsv(datapath, header = true)
headerDict = Dict(col[2] => col[1] for col in enumerate(header))
X = convert(Vector, data[:,headerDict["times"]])
y = convert(Vector, data[:,headerDict["accel"]]);
y /= maximum(y);
scatter(X, y)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -1.5 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

# Construct a region graph

Set some parameters to split up the data using Poon & Domingos Method

In [60]:
# split size
δ = 4

# data range
minX = 0
maxX = 60

60

In [61]:
(rootRegion, sumRegions, gpRegions, allPartitions) = poonDomingos(δ, minX, maxX);

In [62]:
# set IDs for convenients
RegionIDs = Dict(r[2] => r[1] for r in enumerate(union(sumRegions, gpRegions)));
PartitionIDS = Dict(p[2] => p[1] + maximum(values(RegionIDs)) for p in enumerate(allPartitions));

In [63]:
println("Found $(length(sumRegions)) regions with sum nodes")
println("Found $(length(gpRegions)) regions with gp nodes")
println("Found $(length(allPartitions)) partitions with splitting nodes")

Found 105 regions with sum nodes
Found 15 regions with gp nodes
Found 560 partitions with splitting nodes


## Construct SPN from Region Graph

Set some parameters

In [64]:
global gID = 1

numGPs = 10
numSums = 10
meanFunction = MeanZero();
kernelFunction = SE(log(5.0),log(1.0));
noise = -1.;

# put some priors on the parameters
set_priors!(kernelFunction, [Normal(log(5.0),1.0), Normal(log(1.0),1.0)])

2-element Array{Distributions.Normal{Float64},1}:
 Distributions.Normal{Float64}(μ=1.60944, σ=1.0)
 Distributions.Normal{Float64}(μ=0.0, σ=1.0)    

In [65]:
root = convertToSPN(rootRegion, gpRegions, X, y, meanFunction, kernelFunction, noise; sampleNoise = false)

BasicMCJob:
  Variable [1]: p (BasicContMuvParameter)
  GenericModel: 1 variables, 0 dependencies (directed graph)
  HMC sampler: number of leaps = 10, leap step = 0.1
  VanillaMCTuner: period = 100, verbose = false
  BasicMCRange: number of steps = 1100, burnin = 100, thinning = 1BasicMCJob:
  Variable [1]: p (BasicContMuvParameter)
  GenericModel: 1 variables, 0 dependencies (directed graph)
  HMC sampler: number of leaps = 10, leap step = 0.1
  VanillaMCTuner: period = 100, verbose = false
  BasicMCRange: number of steps = 1100, burnin = 100, thinning = 1BasicMCJob:
  Variable [1]: p (BasicContMuvParameter)
  GenericModel: 1 variables, 0 dependencies (directed graph)
  HMC sampler: number of leaps = 10, leap step = 0.1
  VanillaMCTuner: period = 100, verbose = false
  BasicMCRange: number of steps = 1100, burnin = 100, thinning = 1BasicMCJob:
  Variable [1]: p (BasicContMuvParameter)
  GenericModel: 1 variables, 0 dependencies (directed graph)
  HMC sampler: number of leaps = 10, le

Gaussian Process Sum Node [ID: 57752, 
	 w_prior: [0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071], 
	 w_posterior: [0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071, 0.071]]

# Posterior Inference over SPN-GP

In [21]:
function spn_mcmc(spn::GPSumNode, gpNodes::Vector{GPLeaf};
              sampler::Klara.MCSampler=Klara.HMC(),
              nIter::Int = 1000,
              burnin::Int = 0,
              thin::Int = 1)
    
    gps = map(n -> n.gp, gpNodes)
    
    function logpost(hyp::Vector{Float64})  #log-target
        
        for (i, gp) in enumerate(gps)
            p = get_params(gp)
            e = (i*2)
            s = s-1
            p[2:3] = hyp[s:e]
            set_params!(gp, p)
            update_target!(gp)
        end
        
        dirty!(spn)
        spn_update!(spn)
        return spn_posterior(spn)
    end

    function dlogpost(hyp::Vector{Float64}) #gradient of the log-target
        Kgrad_buffer = Array{Float64}(gp.nobsv, gp.nobsv)
        for (i, gp) in enumerate(gps)
            p = get_params(gp)
            e = (i*2)
            s = s-1
            p[2:3] = hyp[s:e]
            set_params!(gp, p)
            update_target_and_dtarget!(gp, noise=false, mean=false, kern=true)
            println(gp.dtarget)
        end
        
        dirty!(spn)
        spn_update!(spn)
        
        return gp.dtarget
    end
    
    start = reduce(vcat, map(gp -> get_params(gp)[2:3], gps))
    starting = Dict(:p=>start)
    q = BasicContMuvParameter(:p, logtarget=logpost, gradlogtarget=dlogpost) 
    model = likelihood_model(q, false)                               #set-up the model
    job = BasicMCJob(model, sampler, BasicMCRange(nsteps=nIter, thinning=thin, burnin=burnin), starting)   #set-up MCMC job
    print(job)                                             #display MCMC set-up for the user
    
    run(job)                          #Run MCMC
    chain = Klara.output(job)         # Extract chain
    #set_params!(gp,start)      #reset the parameters stored in the GP to original values
    return chain
end 

spn_mcmc (generic function with 1 method)

In [49]:
nodes = SumProductNetworks.order(root)

LoadError: [91mInterruptException:[39m

In [23]:
leafs = filter(n -> isa(n, GPLeaf), SumProductNetworks.order(root))

LoadError: [91mInterruptException:[39m

In [22]:
spn_mcmc(root)

LoadError: [91mMethodError: no method matching spn_mcmc(::GPSumNode)[0m
Closest candidates are:
  spn_mcmc(::GPSumNode, [91m::Array{GPLeaf,1}[39m; sampler, nIter, burnin, thin) at In[21]:7[39m

## posterior inference of weights

In [66]:
spn_update!(root)

-44.07970552184925

In [67]:
spn_posterior(root)

-41.264276309734385

In [14]:
#dirty!(root)

In [46]:
root

Gaussian Process Sum Node [ID: 3737, 
	 w_prior: [0.2, 0.2, 0.2, 0.2, 0.2], 
	 w_posterior: [0.2, 0.2, 0.2, 0.2, 0.2]]

## Plotting

In [29]:
x = linspace(minimum(X),maximum(X),S)
r = Dict{Int, SPNGPResult}()
spn_predictIndep(root, x[1], r)

SPNGPResult([-0.0189417, -0.0186206, -0.017108, -0.0146442, -0.0123048, -0.0123971, -0.0164547, -0.00610876, -0.00737654, -0.0152851  …  -0.0152851, -0.0152851, -0.0152851, -0.0152851, -0.0152851, -0.0152851, -0.0152851, -0.0152851, -0.0152851, -0.0152851], [0.146119, 0.146555, 0.143913, 0.144794, 0.161469, 0.165718, 0.151064, 0.142178, 0.155329, 0.146985  …  0.146985, 0.146985, 0.146985, 0.146985, 0.146985, 0.146985, 0.146985, 0.146985, 0.146985, 0.146985], [-3.8271, -3.84091, -3.60772, -3.58972, -4.93948, -5.43163, -4.08455, -3.35093, -4.20372, -3.74326  …  -23.1854, -20.5251, -21.5305, -20.4978, -21.6288, -20.6106, -20.6545, -20.4157, -20.9408, -20.7083])

In [19]:
function spn_predict(root, x::Vector)
    μ = zeros(length(x))
    σ = zeros(length(x))
    
    for (xi, xx) in enumerate(x)
        #println(xi)
        r = Dict{Int, SPNGPResult}()
        spn_predictIndep(root, xx, r)
        μi = r[root.id].mean
        μ2i = r[root.id].meansqr
        σ2i = r[root.id].stdsqr
                
        μ[xi] = μi
        σ[xi] = sqrt(σ2i + μ2i - μi^2)
    end
    return (μ, σ)
end

spn_predict (generic function with 1 method)

In [23]:
x = Float64.(collect(linspace(minimum(X),maximum(X),100)));

In [47]:
(μ, σ) = spn_predict(root, x)

([-0.0148193, -0.0150404, -0.0152613, -0.0154819, -0.0157018, -0.0159209, -0.0161388, -0.0163554, -0.0165705, -0.0167837  …  0.00430179, 0.00457613, 0.00484808, 0.00511684, 0.00538166, 0.00564182, 0.00589664, 0.00614549, 0.00638776, 0.00662292], [0.149591, 0.149288, 0.149005, 0.148744, 0.148502, 0.148281, 0.14808, 0.147898, 0.147735, 0.14759  …  0.152921, 0.15322, 0.153536, 0.153872, 0.154225, 0.154597, 0.154987, 0.155395, 0.155821, 0.156265])

In [68]:
S = 100

#x = linspace(minimum(X),maximum(X),S)
#xx = reduce(hcat, x for k in 1:S)
#yy = reduce(hcat, spn_randIndep(root, x) for k in 1:S)

x = Float64.(collect(linspace(minimum(X),maximum(X),S)));
(μ, σ) = spn_predict(root, x)

gp = GP(reshape(X, 1, length(X)), y, MeanZero(), SE(log(5.0),log(1.0)), -1.)
μgp, σgp = predict_y(gp,linspace(minimum(X),maximum(X),S));

plt = scatter(X, y, label = "observations", legend = false, size = (1024, 768), title = "Univariate Gaussian Moment Matching $(S) samples")
#scatter!(xx+(rand(size(xx)) * 0.3), yy, markersize = 2, markerstrokewidth = 0, label = "Posterior samples from SPN-GP");



plot!(linspace(minimum(X),maximum(X),S), w =4, μgp + 2*σgp, color = :lightgreen, label = "Full GP mean + 2*std")
plot!(linspace(minimum(X),maximum(X),S), w =4, μgp - 2*σgp, color = :lightgreen, label = "Full GP mean - 2*std")
plot!(linspace(minimum(X),maximum(X),S), w =4, μgp, color = :orange, label = "Full GP mean")


plot!(linspace(minimum(X),maximum(X),S), w =2, μ + 2*σ, color = :green, label = "SPN GP mean + 2*std")
plot!(linspace(minimum(X),maximum(X),S), w =2, μ - 2*σ, color = :green, label = "SPN GP mean - 2*std")
plot!(linspace(minimum(X),maximum(X),S), w =2, μ, color = :red, label = "SPN GP mean")

vline!([minimum(map(r -> r.min, gpRegions))], c = :orange)
vline!(map(r -> r.max, gpRegions), c = :orange)

plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 60 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 Univariate Gaussian Moment Matching 100 samples 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip3902)" style="stroke:#90ee90; stroke-width:4; stroke-opacity:1; fill:none" points="
 93.6496,222.852 102.198,227.577 110.746,230.361 119.294,232.156 127.843,233.69 136.391,235.384 144.939,237.335 153.487,239.377 162.035,241.168 170.584,242.309 
 179.132,242.445 187.68,241.356 196.228,239.015 204.777,235.62 213.325,231.6 221.873,227.588 230.421,224.383 238.97,222.877 247.518,223.979 256.066,228.523 
 264.614,237.18 273.162,250.394 281.711,268.33 290.259,290.846 298.807,317.494 307.355,347.534 315.904,379.96 324.452,413.549 333,446.914 341.548,478.584 
 350.097,507.089 358.645,531.062 367.193,549.333 375.741,561.015 384.289,565.557 392.838,562.768 401.386,552.806 409.934,536.14 418.482,513.493 427.031,485.779 
 435.579,454.041 444.127,419.394 452.675,382.977 461.224,345.916 469.772,309.285 478.32,274.077 486.868,241.185 495.416,211.376 503.965,185.284 512.513,163.394 
 521.061,146.037 529.609,133.374 538.158,125.391 546.706,121.887 555.254,122.482 563.802,126.627 572.351,133.641 580.899,142.756 589.447,153.178 597.995,164.142 
 606.543,174.958 615.092,185.048 623.64,193.959 632.188,201.37 640.736,207.09 649.285,211.062 657.833,213.354 666.381,214.163 674.929,213.796 683.478,212.645 
 692.026,211.151 700.574,209.755 709.122,208.847 717.67,208.731 726.219,209.596 734.767,211.509 743.315,214.413 751.863,218.152 760.412,222.485 768.96,227.117 
 777.508,231.728 786.056,236.001 794.605,239.654 803.153,242.468 811.701,244.303 820.249,245.105 828.797,244.904 837.346,243.8 845.894,241.939 854.442,239.495 
 862.99,236.642 871.539,233.532 880.087,230.267 888.635,226.874 897.183,223.27 905.732,219.242 914.28,214.425 922.828,208.318 931.376,200.315 939.924,189.777 
 
 "/>
<polyline clip-path="url(#clip3902)" style="stroke:#90ee90; stroke-width:4; stroke-opacity:1; fill:none" points="
 93.6496,363.513 102.198,359.665 110.746,358.694 119.294,359.605 127.843,361.503 136.391,363.672 144.939,365.595 153.487,366.942 162.035,367.516 170.584,367.218 
 179.132,366.011 187.68,363.918 196.228,361.036 204.777,357.557 213.325,353.799 221.873,350.216 230.421,347.402 238.97,346.065 247.518,346.99 256.066,350.978 
 264.614,358.77 273.162,370.957 281.711,387.892 290.259,409.612 298.807,435.785 307.355,465.7 315.904,498.298 324.452,532.241 333,566.016 341.548,598.052 
 350.097,626.828 358.645,650.978 367.193,669.362 375.741,681.123 384.289,685.722 392.838,682.952 401.386,672.936 409.934,656.11 418.482,633.191 427.031,605.121 
 435.579,573.006 444.127,538.043 452.675,501.457 461.224,464.434 469.772,428.075 478.32,393.369 486.868,361.164 495.416,332.158 503.965,306.896 512.513,285.764 
 521.061,268.993 529.609,256.661 538.158,248.711 546.706,244.956 555.254,245.094 563.802,248.717 572.351,255.319 580.899,264.299 589.447,274.969 597.995,286.581 
 606.543,298.358 615.092,309.552 623.64,319.505 632.188,327.71 640.736,333.859 649.285,337.862 657.833,339.847 666.381,340.121 674.929,339.131 683.478,337.4 
 692.026,335.473 700.574,333.87 709.122,333.037 717.67,333.313 726.219,334.898 734.767,337.843 743.315,342.044 751.863,347.262 760.412,353.155 768.96,359.32 
 777.508,365.346 786.056,370.854 794.605,375.534 803.153,379.164 811.701,381.623 820.249,382.879 828.797,382.986 837.346,382.06 845.894,380.264 854.442,377.78 
 862.99,374.8 871.539,371.506 880.087,368.08 888.635,364.72 897.183,361.668 905.732,3

In [69]:
savefig("../plots/poonDomingos_sharedNoiseModel_moments_large.png")

## Histogram over parameters

## Graph Plotting

In [44]:
sources = Int[]
destinations = Int[]

# add edges from regions to partitions
for r in filter(r -> isa(r, SumRegion), union(sumRegions, gpRegions))
    for p in r.partitions
        push!(sources, RegionIDs[r])
        push!(destinations, PartitionIDS[p])
    end
end

# add edges from partitions to regions
for p in allPartitions
    for r in p.regions
        push!(sources, PartitionIDS[p])
        push!(destinations, RegionIDs[r])
    end
end

In [45]:
nodenames = vcat(map(r -> string("+"),  union(sumRegions, gpRegions)), map(r -> string("*"), allPartitions));
colorings = vcat(map(r -> isa(r, SumRegion) ? :lightblue : :lightgreen,  union(sumRegions, gpRegions)), map(r -> :black, allPartitions));

In [47]:
#gr(size = (400, 400))
#graphplot(sources, destinations, names = nodenames, method = :tree, fontsize = 7, nodesize = 3, m = colorings, nodeshape = :rect)